<a href="https://colab.research.google.com/github/hosseinmiripy/bottelegram/blob/main/photoresizerbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-telegram-bot==13.15 Pillow opencv-python-headless numpy


In [2]:
import getpass

def get_bot_token():
    """دریافت توکن ربات از کاربر"""
    print("لطفاً توکن ربات خود را که از BotFather دریافت کرده‌اید وارد کنید:")
    return getpass.getpass()

def setup_config():
    """تنظیم اولیه ربات"""
    # دریافت توکن ربات
    token = get_bot_token()
    config.set_token(token)

    # دریافت آیدی ادمین اصلی
    print("\nلطفاً آیدی عددی خود را به عنوان ادمین اصلی وارد کنید:")
    print("(برای پیدا کردن آیدی عددی خود می‌توانید به @userinfobot پیام دهید)")
    try:
        admin_id = int(input("آیدی ادمین: "))
        config.add_admin(admin_id)
        print(f"ادمین با آیدی {admin_id} اضافه شد.")
    except ValueError:
        print("آیدی وارد شده صحیح نیست. ادمین به صورت خودکار تنظیم خواهد شد.")

    # دریافت کانال‌های اطلاع‌رسانی
    print("\nآیا می‌خواهید کانال‌های اطلاع‌رسانی را اضافه کنید؟ (بله/خیر)")
    add_channels = input().strip().lower()

    if add_channels in ["بله", "yes", "y", "آره"]:
        print("لطفاً آیدی کانال‌ها را وارد کنید (هر کانال در یک خط):")
        print("مثال: @mychannel")
        print("برای پایان دادن، یک خط خالی وارد کنید:")

        while True:
            channel = input().strip()
            if not channel:
                break

            # اطمینان از اینکه آیدی کانال با @ شروع می‌شود
            if not channel.startswith('@'):
                channel = '@' + channel

            config.add_info_channel(channel)
            print(f"کانال {channel} اضافه شد.")

    print("\nتنظیمات با موفقیت انجام شد.")
    return config

In [ ]:
# PhotoResizer Bot for Telegram - نسخه بهبود یافته
# با پشتیبانی از بررسی عضویت کانال‌ها و امکان بازگشت در هر مرحله

# نصب کتابخانه‌های مورد نیاز
!pip install python-telegram-bot==13.15 Pillow opencv-python-headless numpy

from PIL import Image, ImageDraw, ImageFont
import io
import os
import time
import logging
import json
import io
import re
import getpass
import numpy as np
from PIL import Image
from typing import Dict, List, Union, Optional, Tuple

from telegram import (
    Update, Bot, ParseMode, InlineKeyboardButton, InlineKeyboardMarkup,
    ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove, ChatAction
)
from telegram.ext import (
    Updater, CommandHandler, MessageHandler, Filters,
    CallbackContext, CallbackQueryHandler, ConversationHandler
)

# تنظیم لاگینگ
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)


# کلاس کانفیگ برای تنظیمات ربات
class BotConfig:
    def __init__(self):
        # تنظیمات پیش‌فرض
        self.TOKEN = ""  # توکن ربات تلگرام
        self.ADMIN_IDS = []  # لیست آیدی‌های ادمین‌ها
        self.INFO_CHANNELS = []  # لیست کانال‌های اطلاع‌رسانی (برای نمایش به کاربر)

        # تنظیمات پردازش تصویر
        self.MAX_IMAGE_DIMENSION = 20000  # حداکثر ابعاد مجاز تصویر
        self.MIN_IMAGE_DIMENSION = 10  # حداقل ابعاد مجاز تصویر
        self.MAX_IMAGE_SIZE_MB = 30  #
        Image.MAX_IMAGE_PIXELS = None  # غیرفعال کردن محدودیت اندازه پیلو
        # تنظیمات کیفیت
        self.QUALITY_OPTIONS = {
            "بسیار بالا (95%)": 95,
            "بالا (85%)": 85,
            "متوسط (70%)": 70,
            "کم (50%)": 50,
            "بسیار کم (30%)": 30
        }

        # پسوندهای فایل تصویری مجاز
        self.ALLOWED_IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.webp', '.tiff']

        # MIME types تصاویر مجاز
        self.ALLOWED_IMAGE_MIME_TYPES = [
            'image/jpeg', 'image/png', 'image/bmp', 'image/webp', 'image/tiff'
        ]

    def set_token(self, token):
        self.TOKEN = token

    def add_admin(self, admin_id):
        if admin_id not in self.ADMIN_IDS:
            self.ADMIN_IDS.append(admin_id)

    def add_info_channel(self, channel_id):
        if channel_id not in self.INFO_CHANNELS:
            self.INFO_CHANNELS.append(channel_id)

    def get_info_channels_keyboard(self):
        """تولید کیبورد برای کانال‌های اطلاع‌رسانی"""
        from telegram import InlineKeyboardButton, InlineKeyboardMarkup

        keyboard = []
        for channel in self.INFO_CHANNELS:
            channel_username = channel.replace('@', '')
            keyboard.append([InlineKeyboardButton(f"عضویت در {channel}", url=f"https://t.me/{channel_username}")])

        return InlineKeyboardMarkup(keyboard) if keyboard else None

    def get_quality_keyboard(self, include_custom=True, include_back=True):
        """تولید کیبورد برای انتخاب کیفیت"""
        from telegram import InlineKeyboardButton, InlineKeyboardMarkup

        keyboard = []
        for quality_name in self.QUALITY_OPTIONS:
            quality_value = self.QUALITY_OPTIONS[quality_name]
            keyboard.append([InlineKeyboardButton(quality_name, callback_data=f"quality_{quality_value}")])

        # افزودن گزینه وارد کردن عدد دلخواه
        if include_custom:
            keyboard.append(
                [InlineKeyboardButton("وارد کردن عدد کیفیت دلخواه (1-100)", callback_data="quality_custom")])

        # افزودن گزینه بازگشت
        if include_back:
            keyboard.append([InlineKeyboardButton("بازگشت به روش‌های تغییر اندازه", callback_data="back_to_methods")])

        # افزودن دکمه شروع مجدد
        keyboard.append([InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")])

        return InlineKeyboardMarkup(keyboard)

    def get_info_channels_text(self):
        """متن معرفی کانال‌های اطلاع‌رسانی"""
        if not self.INFO_CHANNELS:
            return ""

        text = "📢 کانال‌های اطلاع‌رسانی ما:\n\n"
        for i, channel in enumerate(self.INFO_CHANNELS, 1):
            text += f"{i}. {channel}\n"

        return text

    def get_membership_verification_keyboard(self, non_member_channels):
        """تولید کیبورد برای بررسی عضویت در کانال‌ها"""
        keyboard = []

        # افزودن دکمه‌های عضویت در کانال‌ها
        for channel in non_member_channels:
            channel_username = channel.replace('@', '')
            keyboard.append([InlineKeyboardButton(f"عضویت در {channel}", url=f"https://t.me/{channel_username}")])

        # افزودن دکمه بررسی عضویت
        keyboard.append([InlineKeyboardButton("✅ عضو شدم، بررسی کن", callback_data="verify_membership")])

        return InlineKeyboardMarkup(keyboard)


# ایجاد نمونه کانفیگ
config = BotConfig()

# ساختار داده‌های مورد نیاز ربات
user_data_store = {}  # برای ذخیره اطلاعات کاربران
user_images = {}  # برای ذخیره موقت تصاویر
bot_statistics = {
    "total_users": 0,
    "processed_images": 0,
}

# تعریف مراحل مختلف برای ConversationHandler
(
    START, WAITING_MEMBERSHIP, RECEIVE_PHOTO, SELECT_RESIZE_METHOD,
    ENTER_WIDTH, ENTER_HEIGHT, ENTER_PERCENTAGE,
    SELECT_QUALITY, ENTER_CUSTOM_QUALITY, CONFIRM_SEND,
    ADMIN_ADD_CHANNEL, ADMIN_REMOVE_CHANNEL, ADMIN_SET_ADMIN
) = range(13)


# توابع تنظیم اولیه ربات
def get_bot_token():
    """دریافت توکن ربات از کاربر"""
    print("لطفاً توکن ربات خود را که از BotFather دریافت کرده‌اید وارد کنید:")
    return getpass.getpass()


def setup_config():
    """تنظیم اولیه ربات"""
    # دریافت توکن ربات
    token = get_bot_token()
    config.set_token(token)

    # دریافت آیدی ادمین اصلی
    print("\nلطفاً آیدی عددی خود را به عنوان ادمین اصلی وارد کنید:")
    print("(برای پیدا کردن آیدی عددی خود می‌توانید به @userinfobot پیام دهید)")
    try:
        admin_id = int(input("آیدی ادمین: "))
        config.add_admin(admin_id)
        print(f"ادمین با آیدی {admin_id} اضافه شد.")
    except ValueError:
        print("آیدی وارد شده صحیح نیست. ادمین به صورت خودکار تنظیم خواهد شد.")

    # دریافت کانال‌های اطلاع‌رسانی
    print("\nآیا می‌خواهید کانال‌های اطلاع‌رسانی را اضافه کنید؟ (بله/خیر)")
    add_channels = input().strip().lower()

    if add_channels in ["بله", "yes", "y", "آره"]:
        print("لطفاً آیدی کانال‌ها را وارد کنید (هر کانال در یک خط):")
        print("مثال: @mychannel")
        print("برای پایان دادن، یک خط خالی وارد کنید:")

        while True:
            channel = input().strip()
            if not channel:
                break

            # اطمینان از اینکه آیدی کانال با @ شروع می‌شود
            if not channel.startswith('@'):
                channel = '@' + channel

            config.add_info_channel(channel)
            print(f"کانال {channel} اضافه شد.")

    print("\nتنظیمات با موفقیت انجام شد.")
    return config


# توابع بررسی عضویت در کانال‌ها
def check_user_membership(bot, user_id, channel):
    """بررسی عضویت کاربر در کانال"""
    try:
        channel_id = channel
        if channel.startswith('@'):
            channel_id = channel
        member = bot.get_chat_member(chat_id=channel_id, user_id=user_id)
        return member.status in ['member', 'administrator', 'creator']
    except Exception as e:
        logger.error(f"خطا در بررسی عضویت: {e}")
        return False


def get_non_member_channels(bot, user_id, channels):
    """دریافت لیست کانال‌هایی که کاربر عضو آنها نیست"""
    non_member_channels = []
    for channel in channels:
        if not check_user_membership(bot, user_id, channel):
            non_member_channels.append(channel)
    return non_member_channels


# توابع پردازش تصویر
def is_valid_image(image: Image.Image) -> bool:
    """بررسی معتبر بودن تصویر از نظر ابعاد و اندازه"""
    width, height = image.size

    # بررسی ابعاد
    if width > config.MAX_IMAGE_DIMENSION or height > config.MAX_IMAGE_DIMENSION:
        return False

    if width < config.MIN_IMAGE_DIMENSION or height < config.MIN_IMAGE_DIMENSION:
        return False

    # بررسی حجم
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='JPEG', quality=95)
    size_bytes = img_byte_arr.tell()

    if size_bytes > config.MAX_IMAGE_SIZE_MB * 1024 * 1024:
        return False

    return True


def is_image_file(file_name: str, mime_type: str = None) -> bool:
    """بررسی اینکه آیا فایل، تصویر است یا خیر"""
    # بررسی پسوند فایل
    ext = os.path.splitext(file_name.lower())[1]
    if ext in config.ALLOWED_IMAGE_EXTENSIONS:
        return True

    # بررسی MIME type
    if mime_type and mime_type in config.ALLOWED_IMAGE_MIME_TYPES:
        return True

    return False


def resize_image_with_aspect_ratio(image: Image.Image, width: Optional[int] = None, height: Optional[int] = None,
                                   keep_aspect: bool = True) -> Image.Image:
    """تغییر اندازه تصویر با حفظ نسبت تصویر"""
    original_width, original_height = image.size

    # اگر هر دو بعد معین باشند
    if width and height:
        if keep_aspect:
            # محاسبه نسبت ابعاد اصلی
            original_ratio = original_width / original_height

            # محاسبه نسبت ابعاد درخواستی
            requested_ratio = width / height

            # اگر نسبت‌ها متفاوت باشند، نسبت را حفظ می‌کنیم
            if abs(original_ratio - requested_ratio) > 0.01:  # مقدار کوچک برای مقایسه اعداد اعشاری
                # اگر نسبت اصلی بزرگتر باشد (تصویر عریض‌تر است)
                if original_ratio > requested_ratio:
                    # عرض را ثابت نگه می‌داریم و ارتفاع را محاسبه می‌کنیم
                    height = int(width / original_ratio)
                else:
                    # ارتفاع را ثابت نگه می‌داریم و عرض را محاسبه می‌کنیم
                    width = int(height * original_ratio)

        return image.resize((width, height), Image.LANCZOS)

    # اگر فقط عرض معین باشد
    if width:
        # محاسبه ارتفاع متناسب
        aspect_ratio = original_height / original_width
        new_height = int(aspect_ratio * width)
        return image.resize((width, new_height), Image.LANCZOS)

    # اگر فقط ارتفاع معین باشد
    if height:
        # محاسبه عرض متناسب
        aspect_ratio = original_width / original_height
        new_width = int(aspect_ratio * height)
        return image.resize((new_width, height), Image.LANCZOS)

    # اگر هیچ بعدی معین نباشد، تصویر اصلی را برمی‌گردانیم
    return image


def resize_by_percentage(image: Image.Image, percentage: int) -> Image.Image:
    """تغییر اندازه تصویر براساس درصد با حفظ نسبت تصویر"""
    if percentage <= 0:
        percentage = 1

    width, height = image.size
    new_width = int(width * percentage / 100)
    new_height = int(height * percentage / 100)

    # اعمال محدودیت‌های ابعاد
    new_width = min(new_width, config.MAX_IMAGE_DIMENSION)
    new_height = min(new_height, config.MAX_IMAGE_DIMENSION)

    new_width = max(new_width, config.MIN_IMAGE_DIMENSION)
    new_height = max(new_height, config.MIN_IMAGE_DIMENSION)

    return image.resize((new_width, new_height), Image.LANCZOS)


def compress_image(image: Image.Image, quality: int) -> bytes:
    """فشرده‌سازی تصویر با کیفیت مشخص شده"""
    img_byte_arr = io.BytesIO()

    # محدود کردن کیفیت به بازه 1-100
    quality = max(1, min(100, quality))

    # تشخیص فرمت تصویر
    if image.mode == 'RGBA':
        # تصاویر با کانال آلفا را به PNG تبدیل می‌کنیم
        image.save(img_byte_arr, format='PNG', optimize=True)
    else:
        # سایر تصاویر را به JPEG تبدیل می‌کنیم
        image.save(img_byte_arr, format='JPEG', quality=quality, optimize=True)

    img_byte_arr.seek(0)
    return img_byte_arr.getvalue()


def get_image_info(image: Image.Image) -> str:
    """اطلاعات تصویر را برمی‌گرداند"""
    width, height = image.size

    # شناسایی فرمت تصویر
    format_name = image.format if image.format else "نامشخص"

    # محاسبه حجم تصویر
    img_byte_arr = io.BytesIO()

    if image.mode == 'RGBA':
        image.save(img_byte_arr, format='PNG')
    else:
        image.save(img_byte_arr, format='JPEG', quality=85)

    size_bytes = img_byte_arr.tell()

    if size_bytes < 1024:
        size_str = f"{size_bytes} بایت"
    elif size_bytes < 1024 * 1024:
        size_str = f"{size_bytes / 1024:.1f} کیلوبایت"
    else:
        size_str = f"{size_bytes / (1024 * 1024):.1f} مگابایت"

    # بررسی حالت تصویر
    mode_info = ""
    if image.mode == 'RGBA':
        mode_info = "با شفافیت (آلفا)"
    elif image.mode == 'RGB':
        mode_info = "RGB"
    else:
        mode_info = f"حالت: {image.mode}"

    return f"📋 اطلاعات تصویر:\n" \
           f"🔹 ابعاد: {width}×{height} پیکسل\n" \
           f"🔹 فرمت: {format_name}\n" \
           f"🔹 نوع: {mode_info}\n" \
           f"🔹 حجم تقریبی: {size_str}"


def optimize_image_for_telegram(image: Image.Image) -> Image.Image:
    """بهینه‌سازی تصویر برای ارسال در تلگرام"""
    # اگر تصویر خیلی بزرگ باشد، آن را کوچک می‌کنیم
    width, height = image.size

    if width > 1280 or height > 1280:
        # کاهش اندازه به حداکثر 1280 پیکسل در هر بعد
        if width >= height:
            new_width = 1280
            new_height = int(height * (1280 / width))
        else:
            new_height = 1280
            new_width = int(width * (1280 / height))

        image = image.resize((new_width, new_height), Image.LANCZOS)

    return image


# هندلرهای ربات
def is_admin(user_id: int) -> bool:
    """بررسی ادمین بودن کاربر"""
    return user_id in config.ADMIN_IDS


def set_first_admin(update: Update):
    """تنظیم اولین ادمین اگر لیست ادمین‌ها خالی باشد"""
    if not config.ADMIN_IDS:
        config.add_admin(update.effective_user.id)
        return True
    return False


def start(update: Update, context: CallbackContext) -> int:
    """هندلر دستور /start"""
    user = update.effective_user
    user_id = user.id

    # تنظیم اولین ادمین
    if set_first_admin(update):
        update.message.reply_text("👑 شما به عنوان اولین ادمین بات تنظیم شدید!")

    # اضافه کردن کاربر به آمار
    if user_id not in user_data_store:
        user_data_store[user_id] = {
            'username': user.username,
            'first_name': user.first_name,
            'last_name': user.last_name,
            'join_date': time.time()
        }
        bot_statistics["total_users"] += 1

    # بررسی عضویت در کانال‌ها
    if config.INFO_CHANNELS:
        non_member_channels = get_non_member_channels(context.bot, user_id, config.INFO_CHANNELS)

        if non_member_channels:
            channels_text = "\n".join([f"• {channel}" for channel in non_member_channels])

            update.message.reply_text(
                f"👋 سلام {user.first_name}!\n\n"
                f"برای استفاده از ربات، لطفاً عضو کانال‌های زیر شوید:\n\n"
                f"{channels_text}\n\n"
                "پس از عضویت، دکمه زیر را بزنید.",
                reply_markup=config.get_membership_verification_keyboard(non_member_channels)
            )
            return WAITING_MEMBERSHIP

    # پیام خوش‌آمدگویی
    welcome_text = f"سلام {user.first_name}! 👋\n\n" \
                   "به ربات تغییر اندازه تصاویر خوش آمدید.\n\n" \
                   "با این ربات می‌توانید:\n" \
                   "🔸 اندازه تصاویر را تغییر دهید\n" \
                   "🔸 کیفیت تصاویر را تنظیم کنید\n" \
                   "🔸 حجم تصاویر را کاهش دهید\n\n" \
                   "لطفاً یک تصویر را برای من ارسال کنید یا آپلود کنید تا اندازه آن را تغییر دهم."

    update.message.reply_text(welcome_text)
    return RECEIVE_PHOTO


def verify_membership(update: Update, context: CallbackContext) -> int:
    """بررسی عضویت کاربر در کانال‌ها"""
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    # بررسی عضویت در کانال‌ها
    non_member_channels = get_non_member_channels(context.bot, user_id, config.INFO_CHANNELS)

    if non_member_channels:
        # کاربر هنوز عضو همه کانال‌ها نیست
        channels_text = "\n".join([f"• {channel}" for channel in non_member_channels])

        query.edit_message_text(
            f"❌ شما هنوز عضو همه کانال‌های زیر نیستید:\n\n"
            f"{channels_text}\n\n"
            "پس از عضویت، دکمه زیر را بزنید.",
            reply_markup=config.get_membership_verification_keyboard(non_member_channels)
        )
        return WAITING_MEMBERSHIP
    else:
        # کاربر عضو همه کانال‌ها است
        welcome_text = f"سلام {query.from_user.first_name}! 👋\n\n" \
                       "به ربات تغییر اندازه تصاویر خوش آمدید.\n\n" \
                       "با این ربات می‌توانید:\n" \
                       "🔸 اندازه تصاویر را تغییر دهید\n" \
                       "🔸 کیفیت تصاویر را تنظیم کنید\n" \
                       "🔸 حجم تصاویر را کاهش دهید\n\n" \
                       "لطفاً یک تصویر را برای من ارسال کنید یا آپلود کنید تا اندازه آن را تغییر دهم."

        query.edit_message_text(welcome_text)
        return RECEIVE_PHOTO


def handle_image_file(update: Update, context: CallbackContext, file_id: str, file_name: str = None) -> int:
    """پردازش تصویر آپلود شده به صورت فایل یا عکس"""
    user_id = update.effective_user.id

    try:
        context.bot.send_chat_action(
            chat_id=update.effective_chat.id,
            action=ChatAction.TYPING
        )

        # دریافت فایل
        file = context.bot.get_file(file_id)
        photo_bytes = io.BytesIO()
        file.download(out=photo_bytes)
        photo_bytes.seek(0)

        # باز کردن تصویر با پیلو
        try:
            image = Image.open(photo_bytes)
        except Exception as e:
            logger.error(f"خطا در باز کردن تصویر: {e}")
            update.message.reply_text(
                "❌ فایل ارسالی قابل پردازش نیست. لطفاً یک تصویر معتبر ارسال کنید."
            )
            return RECEIVE_PHOTO

        # ذخیره فرمت اصلی تصویر
        original_format = image.format

        # بررسی ابعاد تصویر
        if not is_valid_image(image):
            update.message.reply_text(
                f"⚠️ تصویر شما خیلی بزرگ یا خیلی کوچک است!\n\n"
                f"حداکثر ابعاد مجاز: {config.MAX_IMAGE_DIMENSION}×{config.MAX_IMAGE_DIMENSION} پیکسل\n"
                f"حداقل ابعاد مجاز: {config.MIN_IMAGE_DIMENSION}×{config.MIN_IMAGE_DIMENSION} پیکسل\n"
                f"حداکثر حجم مجاز: {config.MAX_IMAGE_SIZE_MB} مگابایت\n\n"
                "لطفاً تصویر دیگری ارسال کنید."
            )
            return RECEIVE_PHOTO

        # ذخیره تصویر در حافظه
        user_images[user_id] = {
            'original_image': image,
            'current_image': image,
            'original_size': image.size,
            'original_format': original_format,
            'file_name': file_name
        }

        # نمایش اطلاعات تصویر و گزینه‌های تغییر اندازه
        width, height = image.size

        keyboard = [
            [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
            [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
            [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        update.message.reply_text(
            f"✅ تصویر شما دریافت شد!\n\n"
            f"{get_image_info(image)}\n\n"
            "لطفاً روش تغییر اندازه را انتخاب کنید:",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return SELECT_RESIZE_METHOD

    except Exception as e:
        logger.error(f"خطا در دریافت تصویر: {e}")
        update.message.reply_text(
            "❌ خطایی رخ داد. لطفاً تصویر دیگری ارسال کنید یا مجدداً تلاش کنید."
        )
        return RECEIVE_PHOTO


def receive_photo(update: Update, context: CallbackContext) -> int:
    """هندلر دریافت تصویر"""
    # بررسی عضویت در کانال‌ها (برای اطمینان)
    if config.INFO_CHANNELS:
        user_id = update.effective_user.id
        non_member_channels = get_non_member_channels(context.bot, user_id, config.INFO_CHANNELS)

        if non_member_channels:
            channels_text = "\n".join([f"• {channel}" for channel in non_member_channels])

            update.message.reply_text(
                f"⚠️ برای استفاده از ربات، لطفاً ابتدا عضو کانال‌های زیر شوید:\n\n"
                f"{channels_text}\n\n"
                "پس از عضویت، دکمه زیر را بزنید.",
                reply_markup=config.get_membership_verification_keyboard(non_member_channels)
            )
            return WAITING_MEMBERSHIP

    # اگر عکس ارسال شده باشد
    if update.message.photo:
        return handle_image_file(update, context, update.message.photo[-1].file_id)

    # اگر فایل ارسال شده باشد
    elif update.message.document:
        document = update.message.document
        file_name = document.file_name
        mime_type = document.mime_type

        # بررسی اینکه آیا فایل، تصویر است
        if is_image_file(file_name, mime_type):
            return handle_image_file(update, context, document.file_id, file_name)
        else:
            update.message.reply_text(
                "❌ فایل ارسالی یک تصویر نیست. لطفاً یک تصویر ارسال کنید."
            )
            return RECEIVE_PHOTO

    # اگر پیام متنی باشد
    else:
        update.message.reply_text(
            "⚠️ لطفاً یک تصویر ارسال کنید یا آپلود کنید."
        )
        return RECEIVE_PHOTO


def select_resize_method(update: Update, context: CallbackContext) -> int:
    """هندلر انتخاب روش تغییر اندازه"""
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    choice = query.data

    if choice == "start_over":
        # بازگشت به شروع
        query.edit_message_text(
            "🔄 لطفاً یک تصویر جدید ارسال کنید."
        )
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    if choice == "resize_dimensions":
        original_width, original_height = user_images[user_id]['original_size']
        query.edit_message_text(
            f"📏 اندازه فعلی تصویر: {original_width}×{original_height} پیکسل\n\n"
            "لطفاً عرض جدید تصویر را به پیکسل وارد کنید.\n"
            "(برای حفظ نسبت تصویر، می‌توانید عدد 0 را وارد کنید)"
        )
        return ENTER_WIDTH

    elif choice == "resize_percentage":
        query.edit_message_text(
            "📐 لطفاً درصد تغییر اندازه را وارد کنید (بین 1 تا 200):\n"
            "مثال: 50 (برای نصف کردن اندازه)\n"
            "مثال: 200 (برای دو برابر کردن اندازه)"
        )
        return ENTER_PERCENTAGE

    elif choice == "compress_only":
        # مستقیماً به مرحله انتخاب کیفیت می‌رویم
        query.edit_message_text(
            "🔄 لطفاً کیفیت فشرده‌سازی را انتخاب کنید:\n\n"
            "توجه: کیفیت بالاتر، حجم فایل بیشتری خواهد داشت.",
            reply_markup=config.get_quality_keyboard(include_back=True)
        )
        return SELECT_QUALITY


def enter_width(update: Update, context: CallbackContext) -> int:
    """هندلر دریافت عرض جدید"""
    user_id = update.effective_user.id
    text = update.message.text.strip()

    # بررسی دستور بازگشت
    if text.lower() in ['/start', '/cancel']:
        update.message.reply_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    try:
        width = int(text)

        # اگر عرض 0 باشد، یعنی کاربر می‌خواهد عرض به صورت خودکار محاسبه شود
        if width == 0:
            context.user_data['resize_width'] = None

            # کیبورد برای بازگشت یا شروع مجدد
            keyboard = [
                [InlineKeyboardButton("🔙 بازگشت", callback_data="back_to_methods")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            update.message.reply_text(
                "عرض به صورت خودکار محاسبه خواهد شد.\n\n"
                "لطفاً ارتفاع جدید تصویر را به پیکسل وارد کنید.\n"
                "(برای حفظ نسبت تصویر، نمی‌توانید هم عرض و هم ارتفاع را 0 وارد کنید)",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return ENTER_HEIGHT

        # بررسی محدوده‌های مجاز
        if width < config.MIN_IMAGE_DIMENSION or width > config.MAX_IMAGE_DIMENSION:
            update.message.reply_text(
                f"❌ عرض وارد شده خارج از محدوده مجاز است.\n"
                f"حداقل: {config.MIN_IMAGE_DIMENSION} پیکسل\n"
                f"حداکثر: {config.MAX_IMAGE_DIMENSION} پیکسل\n\n"
                "لطفاً عرض دیگری وارد کنید:"
            )
            return ENTER_WIDTH

        # ذخیره عرض
        context.user_data['resize_width'] = width

        # کیبورد برای بازگشت یا شروع مجدد
        keyboard = [
            [InlineKeyboardButton("🔙 بازگشت", callback_data="back_to_methods")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        # درخواست ارتفاع
        update.message.reply_text(
            f"عرض جدید: {width} پیکسل\n\n"
            "لطفاً ارتفاع جدید تصویر را به پیکسل وارد کنید.\n"
            "(برای حفظ نسبت تصویر، می‌توانید عدد 0 را وارد کنید)",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return ENTER_HEIGHT

    except ValueError:
        update.message.reply_text(
            "❌ لطفاً یک عدد صحیح وارد کنید."
        )
        return ENTER_WIDTH


def enter_height(update: Update, context: CallbackContext) -> int:
    """هندلر دریافت ارتفاع جدید"""
    user_id = update.effective_user.id

    # اگر پیام متنی نباشد (مثلاً دکمه بازگشت زده شده باشد)
    if update.callback_query:
        query = update.callback_query
        query.answer()

        if query.data == "back_to_methods":
            # بازگشت به انتخاب روش
            keyboard = [
                [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
                [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
                [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            query.edit_message_text(
                "لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return SELECT_RESIZE_METHOD

        elif query.data == "start_over":
            # شروع مجدد
            query.edit_message_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
            # پاک کردن داده‌های تصویر کاربر
            if user_id in user_images:
                del user_images[user_id]
            return RECEIVE_PHOTO

        return ENTER_HEIGHT

    text = update.message.text.strip()

    # بررسی دستور بازگشت
    if text.lower() in ['/start', '/cancel']:
        update.message.reply_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    try:
        height = int(text)
        width = context.user_data.get('resize_width')

        # اگر ارتفاع 0 باشد، یعنی کاربر می‌خواهد ارتفاع به صورت خودکار محاسبه شود
        if height == 0:
            height = None

        # بررسی اینکه حداقل یکی از ابعاد مشخص باشد
        if width is None and height is None:
            update.message.reply_text(
                "❌ نمی‌توانید هم عرض و هم ارتفاع را 0 وارد کنید. حداقل یکی باید مشخص باشد.\n\n"
                "لطفاً ارتفاع دیگری وارد کنید:"
            )
            return ENTER_HEIGHT

        # بررسی محدوده‌های مجاز
        if height is not None and (height < config.MIN_IMAGE_DIMENSION or height > config.MAX_IMAGE_DIMENSION):
            update.message.reply_text(
                f"❌ ارتفاع وارد شده خارج از محدوده مجاز است.\n"
                f"حداقل: {config.MIN_IMAGE_DIMENSION} پیکسل\n"
                f"حداکثر: {config.MAX_IMAGE_DIMENSION} پیکسل\n\n"
                "لطفاً ارتفاع دیگری وارد کنید:"
            )
            return ENTER_HEIGHT

        # تغییر اندازه تصویر
        original_image = user_images[user_id]['original_image']
        original_width, original_height = user_images[user_id]['original_size']

        # کیبورد انتخاب حفظ نسبت تصویر
        # فقط اگر هر دو بعد مشخص شده باشند و نسبت متفاوت باشد
        if width is not None and height is not None:
            original_ratio = original_width / original_height
            new_ratio = width / height

            if abs(original_ratio - new_ratio) > 0.01:
                # نسبت‌ها متفاوت است، پس گزینه حفظ نسبت تصویر را نشان می‌دهیم
                keyboard = [
                    [InlineKeyboardButton("حفظ نسبت تصویر (جلوگیری از کشیدگی)", callback_data="keep_aspect_true")],
                    [InlineKeyboardButton("تغییر به ابعاد دقیق (امکان کشیدگی)", callback_data="keep_aspect_false")],
                    [InlineKeyboardButton("🔙 بازگشت", callback_data="back_to_methods")],
                    [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
                ]

                # تخمین ابعاد حاصل از حفظ نسبت
                if original_ratio > new_ratio:
                    # تصویر عریض‌تر از درخواست است
                    adjusted_height = int(width / original_ratio)
                    aspect_text = f"با حفظ نسبت: {width}×{adjusted_height}"
                else:
                    # تصویر بلندتر از درخواست است
                    adjusted_width = int(height * original_ratio)
                    aspect_text = f"با حفظ نسبت: {adjusted_width}×{height}"

                update.message.reply_text(
                    f"⚠️ ابعاد درخواستی ({width}×{height}) با نسبت اصلی تصویر متفاوت است.\n"
                    f"نسبت اصلی: {original_ratio:.2f}\n"
                    f"نسبت درخواستی: {new_ratio:.2f}\n\n"
                    f"{aspect_text}\n\n"
                    "لطفاً یکی از گزینه‌های زیر را انتخاب کنید:",
                    reply_markup=InlineKeyboardMarkup(keyboard)
                )

                # ذخیره ابعاد درخواستی
                context.user_data['resize_height'] = height

                return SELECT_QUALITY

            # تغییر اندازه تصویر با حفظ نسبت تصویر
            resized_image = resize_image_with_aspect_ratio(original_image, width, height)
            user_images[user_id]['current_image'] = resized_image

            # ارائه گزینه‌های کیفیت
            keyboard = config.get_quality_keyboard(include_back=True)

            # افزودن واترمارک به تصویر تغییر اندازه داده شده
            resized_image_with_watermark = add_watermark(resized_image)

            # ارسال پیش‌نمایش تصویر تغییر اندازه داده شده
            preview_bytes = io.BytesIO()

            # استفاده از فرمت اصلی برای پیش‌نمایش اگر ممکن باشد
            preview_format = user_images[user_id].get('original_format', 'JPEG')
            if resized_image_with_watermark.mode == 'RGBA' and preview_format != 'PNG':
                preview_format = 'PNG'

            resized_image_with_watermark.save(preview_bytes, format=preview_format, quality=70)
            preview_bytes.seek(0)

            update.message.bot.send_chat_action(
                chat_id=update.effective_chat.id,
                action=ChatAction.UPLOAD_PHOTO
            )

            actual_width, actual_height = resized_image_with_watermark.size

            # بررسی اندازه تصویر برای تصمیم‌گیری در مورد نحوه ارسال
            if is_preview_too_large(resized_image_with_watermark):
                # ارسال به عنوان فایل
                update.message.bot.send_document(
                    chat_id=update.effective_chat.id,
                    document=preview_bytes,
                    filename=f"preview_{actual_width}x{actual_height}.jpg",
                    caption=f"🖼 پیش‌نمایش تصویر با اندازه {actual_width}×{actual_height} پیکسل\n\n"
                            f"{get_image_info(resized_image)}\n\n"
                            "لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
                    reply_markup=keyboard
                )
            else:
                # ارسال به عنوان عکس
                update.message.bot.send_photo(
                    chat_id=update.effective_chat.id,
                    photo=preview_bytes,
                    caption=f"🖼 پیش‌نمایش تصویر با اندازه {actual_width}×{actual_height} پیکسل\n\n"
                            f"{get_image_info(resized_image)}\n\n"
                            "لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
                    reply_markup=keyboard
                )

            return SELECT_QUALITY

    except ValueError:
        update.message.reply_text(
            "❌ لطفاً یک عدد صحیح وارد کنید."
        )
        return ENTER_HEIGHT


def aspect_ratio_choice(update: Update, context: CallbackContext) -> int:
    """هندلر انتخاب حفظ نسبت تصویر"""
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    choice = query.data

    if choice == "back_to_methods":
        # بازگشت به انتخاب روش
        keyboard = [
            [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
            [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
            [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        query.edit_message_text(
            "لطفاً روش تغییر اندازه را انتخاب کنید:",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return SELECT_RESIZE_METHOD

    elif choice == "start_over":
        # شروع مجدد
        query.edit_message_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    keep_aspect = choice == "keep_aspect_true"

    # بازیابی ابعاد درخواستی
    width = context.user_data.get('resize_width')
    height = context.user_data.get('resize_height')

    # تغییر اندازه تصویر
    original_image = user_images[user_id]['original_image']
    resized_image = resize_image_with_aspect_ratio(original_image, width, height, keep_aspect)
    user_images[user_id]['current_image'] = resized_image

    # ارائه گزینه‌های کیفیت
    keyboard = config.get_quality_keyboard(include_back=True)

    # ارسال پیش‌نمایش تصویر تغییر اندازه داده شده
    preview_bytes = io.BytesIO()

    # استفاده از فرمت اصلی برای پیش‌نمایش اگر ممکن باشد
    preview_format = user_images[user_id].get('original_format', 'JPEG')
    if resized_image.mode == 'RGBA' and preview_format != 'PNG':
        preview_format = 'PNG'

    resized_image.save(preview_bytes, format=preview_format, quality=70)
    preview_bytes.seek(0)

    query.message.reply_photo(
        photo=preview_bytes,
        caption=f"🖼 پیش‌نمایش تصویر با {'حفظ نسبت' if keep_aspect else 'ابعاد دقیق'}\n"
                f"ابعاد: {resized_image.width}×{resized_image.height} پیکسل\n\n"
                f"{get_image_info(resized_image)}\n\n"
                "لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
        reply_markup=keyboard
    )

    return SELECT_QUALITY


def enter_percentage(update: Update, context: CallbackContext) -> int:
    """هندلر دریافت درصد تغییر اندازه"""
    user_id = update.effective_user.id

    # اگر پیام متنی نباشد (مثلاً دکمه بازگشت زده شده باشد)
    if update.callback_query:
        query = update.callback_query
        query.answer()

        if query.data == "back_to_methods":
            # بازگشت به انتخاب روش
            keyboard = [
                [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
                [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
                [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            query.edit_message_text(
                "لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return SELECT_RESIZE_METHOD

        elif query.data == "start_over":
            # شروع مجدد
            query.edit_message_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
            # پاک کردن داده‌های تصویر کاربر
            if user_id in user_images:
                del user_images[user_id]
            return RECEIVE_PHOTO

        return ENTER_PERCENTAGE

    context.bot.send_chat_action(
        chat_id=update.effective_chat.id,
        action=ChatAction.TYPING
    )

    text = update.message.text.strip()

    # بررسی دستور بازگشت
    if text.lower() in ['/start', '/cancel']:
        update.message.reply_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    try:
        percentage = int(text)

        if percentage <= 0 or percentage > 200:
            # کیبورد برای بازگشت یا شروع مجدد
            keyboard = [
                [InlineKeyboardButton("🔙 بازگشت", callback_data="back_to_methods")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            update.message.reply_text(
                "❌ درصد باید بین 1 تا 200 باشد. لطفاً عدد دیگری وارد کنید.",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return ENTER_PERCENTAGE

        # تغییر اندازه تصویر با درصد
        original_image = user_images[user_id]['original_image']
        resized_image = resize_by_percentage(original_image, percentage)
        user_images[user_id]['current_image'] = resized_image

        # ارائه گزینه‌های کیفیت
        keyboard = config.get_quality_keyboard(include_back=True)

        # ارسال پیش‌نمایش تصویر تغییر اندازه داده شده
        preview_bytes = io.BytesIO()

        # استفاده از فرمت اصلی برای پیش‌نمایش اگر ممکن باشد
        preview_format = user_images[user_id].get('original_format', 'JPEG')
        if resized_image.mode == 'RGBA' and preview_format != 'PNG':
            preview_format = 'PNG'

        resized_image.save(preview_bytes, format=preview_format, quality=70)
        preview_bytes.seek(0)

        context.bot.send_chat_action(
            chat_id=update.effective_chat.id,
            action=ChatAction.UPLOAD_PHOTO
        )

        width, height = resized_image.size
        original_width, original_height = user_images[user_id]['original_size']

        context.bot.send_photo(
            chat_id=update.effective_chat.id,
            photo=preview_bytes,
            caption=f"🖼 پیش‌نمایش تصویر با {percentage}% اندازه اصلی\n"
                    f"اندازه اصلی: {original_width}×{original_height}\n"
                    f"اندازه جدید: {width}×{height}\n\n"
                    f"{get_image_info(resized_image)}\n\n"
                    "لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
            reply_markup=keyboard
        )

        return SELECT_QUALITY

    except ValueError:
        # کیبورد برای بازگشت یا شروع مجدد
        keyboard = [
            [InlineKeyboardButton("🔙 بازگشت", callback_data="back_to_methods")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        update.message.reply_text(
            "❌ لطفاً یک عدد صحیح وارد کنید. مثال: 50",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return ENTER_PERCENTAGE
    except Exception as e:
        logger.error(f"خطای عمومی در تغییر اندازه با درصد: {e}")
        update.message.reply_text(
            "❌ خطایی در پردازش تصویر رخ داد. لطفاً دوباره تلاش کنید."
        )
        return ENTER_PERCENTAGE


def select_quality(update: Update, context: CallbackContext) -> int:
    """هندلر انتخاب کیفیت"""
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    choice = query.data

    # بررسی انتخاب روش حفظ نسبت تصویر
    if choice.startswith("keep_aspect_"):
        return aspect_ratio_choice(update, context)

    if choice == "back_to_methods":
        # بازگشت به مرحله انتخاب روش تغییر اندازه
        keyboard = [
            [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
            [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
            [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        try:
            query.edit_message_caption(
                caption=f"لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
        except:
            # اگر پیام عکس نداشت
            query.edit_message_text(
                text=f"لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
        return SELECT_RESIZE_METHOD

    elif choice == "start_over":
        # شروع مجدد
        try:
            query.edit_message_caption(
                caption="🔄 لطفاً یک تصویر جدید ارسال کنید."
            )
        except:
            query.edit_message_text(
                text="🔄 لطفاً یک تصویر جدید ارسال کنید."
            )
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    # انتخاب کیفیت سفارشی
    if choice == "quality_custom":
        try:
            query.edit_message_reply_markup(reply_markup=None)

            # کیبورد برای بازگشت یا شروع مجدد
            keyboard = [
                [InlineKeyboardButton("🔙 بازگشت به انتخاب کیفیت", callback_data="back_to_quality")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            query.message.reply_text(
                "🔢 لطفاً عدد کیفیت دلخواه را وارد کنید (بین 1 تا 100):\n"
                "1: کمترین کیفیت، کمترین حجم\n"
                "100: بیشترین کیفیت، بیشترین حجم",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return ENTER_CUSTOM_QUALITY
        except:
            # کیبورد برای بازگشت یا شروع مجدد
            keyboard = [
                [InlineKeyboardButton("🔙 بازگشت به انتخاب کیفیت", callback_data="back_to_quality")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            query.message.reply_text(
                "🔢 لطفاً عدد کیفیت دلخواه را وارد کنید (بین 1 تا 100):\n"
                "1: کمترین کیفیت، کمترین حجم\n"
                "100: بیشترین کیفیت، بیشترین حجم",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return ENTER_CUSTOM_QUALITY

    # انتخاب کیفیت از پیش تعیین شده
    quality = int(choice.split('_')[1])
    user_images[user_id]['quality'] = quality

    # بدون تغییر ابعاد، فقط فشرده‌سازی با کیفیت انتخاب شده
    current_image = user_images[user_id]['current_image']
    user_images[user_id]['optimized_image'] = current_image  # بدون تغییر در ابعاد

    # فشرده‌سازی با کیفیت انتخاب شده
    compressed_bytes = compress_image(current_image, quality)
    user_images[user_id]['compressed_bytes'] = compressed_bytes

    # محاسبه حجم فایل فشرده شده
    size_bytes = len(compressed_bytes)
    if size_bytes < 1024:
        size_str = f"{size_bytes} بایت"
    elif size_bytes < 1024 * 1024:
        size_str = f"{size_bytes / 1024:.1f} کیلوبایت"
    else:
        size_str = f"{size_bytes / (1024 * 1024):.1f} مگابایت"

    # ارسال پیش‌نمایش نهایی
    width, height = current_image.size

    keyboard = [
        [InlineKeyboardButton("✅ ارسال تصویر نهایی", callback_data="send_final")],
        [InlineKeyboardButton("🔄 تغییر کیفیت", callback_data="change_quality")],
        [InlineKeyboardButton("🔙 بازگشت به روش‌ها", callback_data="back_to_methods")],
        [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")],
    ]

    # یافتن نام کیفیت انتخاب شده
    quality_name = f"کیفیت سفارشی ({quality}%)"
    for name, value in config.QUALITY_OPTIONS.items():
        if value == quality:
            quality_name = name
            break

    # محاسبه درصد فشرده‌سازی
    original_image = user_images[user_id]['original_image']
    original_bytes = io.BytesIO()
    if original_image.mode == 'RGBA':
        original_image.save(original_bytes, format='PNG')
    else:
        original_image.save(original_bytes, format='JPEG', quality=100)

    original_size = original_bytes.tell()
    compression_ratio = (1 - (size_bytes / original_size)) * 100 if original_size > 0 else 0

    try:
        query.edit_message_caption(
            caption=f"✨ پیش‌نمایش نهایی تصویر:\n"
                    f"▫️ ابعاد: {width}×{height} پیکسل\n"
                    f"▫️ کیفیت: {quality_name}\n"
                    f"▫️ حجم: {size_str}\n"
                    f"▫️ میزان فشرده‌سازی: {compression_ratio:.1f}%\n\n"
                    "آیا می‌خواهید این تصویر را دریافت کنید؟",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
    except:
        # اگر پیام عکس نداشت
        query.edit_message_text(
            text=f"✨ پیش‌نمایش نهایی تصویر:\n"
                 f"▫️ ابعاد: {width}×{height} پیکسل\n"
                 f"▫️ کیفیت: {quality_name}\n"
                 f"▫️ حجم: {size_str}\n"
                 f"▫️ میزان فشرده‌سازی: {compression_ratio:.1f}%\n\n"
                 "آیا می‌خواهید این تصویر را دریافت کنید؟",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )

    return CONFIRM_SEND


def enter_custom_quality(update: Update, context: CallbackContext) -> int:
    """هندلر دریافت کیفیت سفارشی"""
    user_id = update.effective_user.id

    # اگر پیام متنی نباشد (مثلاً دکمه بازگشت زده شده باشد)
    if update.callback_query:
        query = update.callback_query
        query.answer()

        if query.data == "back_to_quality":
            # بازگشت به انتخاب کیفیت
            keyboard = config.get_quality_keyboard(include_back=True)

            try:
                query.edit_message_text(
                    text="🔄 لطفاً کیفیت فشرده‌سازی را انتخاب کنید:\n\n"
                         "توجه: کیفیت بالاتر، حجم فایل بیشتری خواهد داشت.",
                    reply_markup=keyboard
                )
            except:
                query.message.reply_text(
                    "🔄 لطفاً کیفیت فشرده‌سازی را انتخاب کنید:\n\n"
                    "توجه: کیفیت بالاتر، حجم فایل بیشتری خواهد داشت.",
                    reply_markup=keyboard
                )
            return SELECT_QUALITY

        elif query.data == "start_over":
            # شروع مجدد
            query.edit_message_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
            # پاک کردن داده‌های تصویر کاربر
            if user_id in user_images:
                del user_images[user_id]
            return RECEIVE_PHOTO

        return ENTER_CUSTOM_QUALITY

    text = update.message.text.strip()

    # بررسی دستور بازگشت
    if text.lower() in ['/start', '/cancel']:
        update.message.reply_text("🔄 لطفاً یک تصویر جدید ارسال کنید.")
        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]
        return RECEIVE_PHOTO

    try:
        quality = int(text)

        if quality < 1 or quality > 100:
            # کیبورد برای بازگشت یا شروع مجدد
            keyboard = [
                [InlineKeyboardButton("🔙 بازگشت به انتخاب کیفیت", callback_data="back_to_quality")],
                [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
            ]

            update.message.reply_text(
                "❌ کیفیت باید بین 1 تا 100 باشد. لطفاً عدد دیگری وارد کنید.",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return ENTER_CUSTOM_QUALITY

        # ذخیره کیفیت
        user_images[user_id]['quality'] = quality

        # بدون تغییر ابعاد، فقط فشرده‌سازی با کیفیت انتخاب شده
        current_image = user_images[user_id]['current_image']
        user_images[user_id]['optimized_image'] = current_image  # بدون تغییر در ابعاد

        # فشرده‌سازی
        compressed_bytes = compress_image(current_image, quality)
        user_images[user_id]['compressed_bytes'] = compressed_bytes

        # محاسبه حجم فایل فشرده شده
        size_bytes = len(compressed_bytes)
        if size_bytes < 1024:
            size_str = f"{size_bytes} بایت"
        elif size_bytes < 1024 * 1024:
            size_str = f"{size_bytes / 1024:.1f} کیلوبایت"
        else:
            size_str = f"{size_bytes / (1024 * 1024):.1f} مگابایت"

        # ارسال پیش‌نمایش تصویر نهایی
        width, height = current_image.size

        keyboard = [
            [InlineKeyboardButton("✅ ارسال تصویر نهایی", callback_data="send_final")],
            [InlineKeyboardButton("🔄 تغییر کیفیت", callback_data="change_quality")],
            [InlineKeyboardButton("🔙 بازگشت به روش‌ها", callback_data="back_to_methods")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")],
        ]

        # محاسبه درصد فشرده‌سازی
        original_image = user_images[user_id]['original_image']
        original_bytes = io.BytesIO()
        if original_image.mode == 'RGBA':
            original_image.save(original_bytes, format='PNG')
        else:
            original_image.save(original_bytes, format='JPEG', quality=100)

        original_size = original_bytes.tell()
        compression_ratio = (1 - (size_bytes / original_size)) * 100 if original_size > 0 else 0

        # ارسال پیش‌نمایش
        preview_bytes = io.BytesIO()

        # استفاده از فرمت اصلی برای پیش‌نمایش اگر ممکن باشد
        preview_format = user_images[user_id].get('original_format', 'JPEG')
        if current_image.mode == 'RGBA' and preview_format != 'PNG':
            preview_format = 'PNG'

        current_image.save(preview_bytes, format=preview_format, quality=quality)
        preview_bytes.seek(0)

        update.message.bot.send_photo(
            chat_id=update.effective_chat.id,
            photo=preview_bytes,
            caption=f"✨ پیش‌نمایش نهایی تصویر با کیفیت سفارشی ({quality}%):\n"
                    f"▫️ ابعاد: {width}×{height} پیکسل\n"
                    f"▫️ حجم: {size_str}\n"
                    f"▫️ میزان فشرده‌سازی: {compression_ratio:.1f}%\n\n"
                    "آیا می‌خواهید این تصویر را دریافت کنید؟",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )

        return CONFIRM_SEND

    except ValueError:
        # کیبورد برای بازگشت یا شروع مجدد
        keyboard = [
            [InlineKeyboardButton("🔙 بازگشت به انتخاب کیفیت", callback_data="back_to_quality")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        update.message.reply_text(
            "❌ لطفاً یک عدد صحیح وارد کنید.",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return ENTER_CUSTOM_QUALITY


def add_watermark(image: Image.Image, text: str = "@ANDISHARIA") -> Image.Image:
    """افزودن واترمارک متنی به تصویر"""
    # کپی از تصویر اصلی
    watermarked_img = image.copy()

    # محاسبه اندازه تصویر و فونت
    width, height = watermarked_img.size
    font_size = max(12, min(width // 30, 36))  # سایز فونت متناسب با تصویر

    # ایجاد یک طرح برای متن
    draw = ImageDraw.Draw(watermarked_img)

    # استفاده از فونت پیش‌فرض
    font = ImageFont.load_default()

    # محاسبه موقعیت متن (گوشه پایین سمت راست)
    margin = 10  # فاصله از لبه‌های تصویر
    position = (width - 120, height - 30)  # موقعیت ثابت برای سادگی

    # اضافه کردن سایه برای خوانایی بهتر
    draw.text((position[0] + 2, position[1] + 2), text, (0, 0, 0), font=font)

    # متن اصلی با رنگ سفید
    draw.text(position, text, (255, 255, 255), font=font)

    return watermarked_img


def is_preview_too_large(image: Image.Image, max_size_bytes: int = 10 * 1024 * 1024) -> bool:
    """بررسی اینکه آیا تصویر برای ارسال به عنوان پیش‌نمایش خیلی بزرگ است"""
    img_byte_arr = io.BytesIO()

    # ذخیره تصویر در حافظه موقت برای محاسبه حجم
    if image.mode == 'RGBA':
        image.save(img_byte_arr, format='PNG')
    else:
        image.save(img_byte_arr, format='JPEG', quality=85)

    # محاسبه حجم
    size_bytes = img_byte_arr.tell()

    # بررسی آیا از حداکثر مجاز بیشتر است
    return size_bytes > max_size_bytes


# این تابع باید در تصویر نهایی هم واترمارک اضافه کند
def prepare_final_image(image: Image.Image, quality: int) -> bytes:
    """آماده‌سازی تصویر نهایی با واترمارک و فشرده‌سازی"""
    # افزودن واترمارک
    image_with_watermark = add_watermark(image)

    # فشرده‌سازی
    img_byte_arr = io.BytesIO()

    # محدود کردن کیفیت به بازه 1-100
    quality = max(1, min(100, quality))

    # تشخیص فرمت تصویر
    if image_with_watermark.mode == 'RGBA':
        # تصاویر با کانال آلفا را به PNG تبدیل می‌کنیم
        image_with_watermark.save(img_byte_arr, format='PNG', optimize=True)
    else:
        # سایر تصاویر را به JPEG تبدیل می‌کنیم
        image_with_watermark.save(img_byte_arr, format='JPEG', quality=quality, optimize=True)

    img_byte_arr.seek(0)
    return img_byte_arr.getvalue()

def confirm_send(update: Update, context: CallbackContext) -> int:
    """هندلر تایید ارسال تصویر نهایی"""
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    choice = query.data

    if choice == "send_final":
        # ارسال تصویر نهایی
        current_image = user_images[user_id]['current_image']
        quality = user_images[user_id]['quality']
        width, height = current_image.size

        context.bot.send_chat_action(
            chat_id=update.effective_chat.id,
            action=ChatAction.UPLOAD_DOCUMENT
        )

        # به‌روزرسانی آمار
        bot_statistics["processed_images"] += 1

        # آماده‌سازی تصویر نهایی با واترمارک و فشرده‌سازی
        compressed_bytes = prepare_final_image(current_image, quality)

        # تعیین پسوند فایل براساس مُد تصویر
        extension = "png" if current_image.mode == 'RGBA' else "jpg"

        # تعیین نام فایل خروجی
        original_file_name = user_images[user_id].get('file_name')
        if original_file_name:
            base_name = os.path.splitext(original_file_name)[0]
            output_file_name = f"{base_name}_resized_{width}x{height}_q{quality}.{extension}"
        else:
            output_file_name = f"resized_{width}x{height}_q{quality}.{extension}"

        # یافتن نام کیفیت انتخاب شده
        quality_name = f"سفارشی ({quality}%)"
        for name, value in config.QUALITY_OPTIONS.items():
            if value == quality:
                quality_name = name
                break

        # ارسال تصویر به عنوان فایل
        sent_message = context.bot.send_document(
            chat_id=update.effective_chat.id,
            document=io.BytesIO(compressed_bytes),
            filename=output_file_name,
            caption=f"✅ تصویر تغییر اندازه داده شده شما:\n"
                    f"▫️ ابعاد: {width}×{height} پیکسل\n"
                    f"▫️ کیفیت: {quality_name}\n\n"
                    "🔄 برای تغییر اندازه تصویر دیگر، لطفاً آن را ارسال کنید."
        )

        # ارسال پیام راهنما
        context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="📱 می‌توانید تصویر دیگری ارسال کنید یا از دستور /start برای شروع مجدد استفاده کنید."
        )

        # پاک کردن داده‌های تصویر کاربر برای آزاد کردن حافظه
        if user_id in user_images:
            del user_images[user_id]

        # پاک کردن داده‌های user_data
        if 'resize_width' in context.user_data:
            del context.user_data['resize_width']
        if 'resize_height' in context.user_data:
            del context.user_data['resize_height']

        return RECEIVE_PHOTO

    elif choice == "change_quality":
        # بازگشت به مرحله انتخاب کیفیت
        keyboard = config.get_quality_keyboard(include_back=True)

        try:
            query.edit_message_caption(
                caption="لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
                reply_markup=keyboard
            )
        except:
            query.edit_message_text(
                text="لطفاً کیفیت فشرده‌سازی را انتخاب کنید:",
                reply_markup=keyboard
            )
        return SELECT_QUALITY

    elif choice == "back_to_methods":
        # بازگشت به مرحله انتخاب روش تغییر اندازه
        keyboard = [
            [InlineKeyboardButton("تغییر اندازه با ابعاد مشخص", callback_data="resize_dimensions")],
            [InlineKeyboardButton("تغییر اندازه با درصد", callback_data="resize_percentage")],
            [InlineKeyboardButton("کاهش حجم بدون تغییر ابعاد", callback_data="compress_only")],
            [InlineKeyboardButton("🔄 شروع مجدد", callback_data="start_over")]
        ]

        try:
            query.edit_message_caption(
                caption="لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
        except:
            query.edit_message_text(
                text="لطفاً روش تغییر اندازه را انتخاب کنید:",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )

        return SELECT_RESIZE_METHOD

    elif choice == "start_over":
        # شروع مجدد از ابتدا
        try:
            query.edit_message_caption(
                caption="🔄 لطفاً یک تصویر جدید ارسال کنید."
            )
        except:
            query.edit_message_text(
                text="🔄 لطفاً یک تصویر جدید ارسال کنید."
            )

        # پاک کردن داده‌های تصویر کاربر
        if user_id in user_images:
            del user_images[user_id]

        # پاک کردن داده‌های user_data
        if 'resize_width' in context.user_data:
            del context.user_data['resize_width']
        if 'resize_height' in context.user_data:
            del context.user_data['resize_height']

        return RECEIVE_PHOTO


# هندلرهای ادمین
def admin_command(update: Update, context: CallbackContext) -> None:
    """هندلر دستور /admin"""
    user_id = update.effective_user.id

    if not is_admin(user_id):
        update.message.reply_text("⛔ شما دسترسی ادمین ندارید.")
        return

    # منوی ادمین
    keyboard = [
        [KeyboardButton("/add_admin - افزودن ادمین")],
        [KeyboardButton("/stats - آمار ربات")],
        [KeyboardButton("/cancel - بازگشت")],
    ]

    update.message.reply_text(
        "👑 پنل مدیریت ربات:\n"
        "لطفاً یکی از گزینه‌های زیر را انتخاب کنید:",
        reply_markup=ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
    )


def add_admin_command(update: Update, context: CallbackContext) -> int:
    """هندلر دستور /add_admin"""
    user_id = update.effective_user.id

    if not is_admin(user_id):
        update.message.reply_text("⛔ شما دسترسی ادمین ندارید.")
        return ConversationHandler.END

    update.message.reply_text(
        "👤 لطفاً آیدی عددی کاربر جدید را وارد کنید یا پیام کاربر را forward کنید.\n"
        "می‌توانید با دستور /cancel عملیات را لغو کنید."
    )

    return ADMIN_SET_ADMIN


def add_admin(update: Update, context: CallbackContext) -> int:
    """هندلر افزودن ادمین"""
    user_id = update.effective_user.id

    if not is_admin(user_id):
        update.message.reply_text("⛔ شما دسترسی ادمین ندارید.")
        return ConversationHandler.END

    # دریافت آیدی کاربر
    if update.message.forward_from:
        # اگر پیام فوروارد شده باشد
        new_admin_id = update.message.forward_from.id
        new_admin_name = update.message.forward_from.first_name
    else:
        # اگر آیدی به صورت متن وارد شده باشد
        try:
            new_admin_id = int(update.message.text.strip())
            new_admin_name = str(new_admin_id)
        except ValueError:
            update.message.reply_text("❌ لطفاً یک آیدی عددی معتبر وارد کنید یا پیام کاربر را forward کنید.")
            return ADMIN_SET_ADMIN

    # افزودن ادمین جدید
    if new_admin_id in config.ADMIN_IDS:
        update.message.reply_text("⚠️ این کاربر قبلاً به عنوان ادمین تنظیم شده است.")
    else:
        config.add_admin(new_admin_id)
        update.message.reply_text(f"✅ کاربر {new_admin_name} با موفقیت به عنوان ادمین اضافه شد.")

    return ConversationHandler.END


def stats_command(update: Update, context: CallbackContext) -> None:
    """هندلر دستور /stats"""
    user_id = update.effective_user.id

    # آمار برای همه کاربران در دسترس است
    stats_text = "📊 آمار ربات:\n\n"
    stats_text += f"👥 تعداد کل کاربران: {bot_statistics['total_users']}\n"
    stats_text += f"🖼 تصاویر پردازش شده: {bot_statistics['processed_images']}\n"

    # اطلاعات بیشتر برای ادمین‌ها
    if is_admin(user_id):
        stats_text += f"\n👮‍♂️ تعداد ادمین‌ها: {len(config.ADMIN_IDS)}\n"
        stats_text += f"🧠 تعداد کاربران فعال در حافظه: {len(user_data_store)}\n"
        stats_text += f"💾 تعداد تصاویر در حال پردازش: {len(user_images)}\n"

    update.message.reply_text(stats_text)


def cancel_command(update: Update, context: CallbackContext) -> int:
    """هندلر دستور /cancel"""
    update.message.reply_text(
        "🔄 عملیات لغو شد. می‌توانید یک تصویر برای تغییر اندازه ارسال کنید.",
        reply_markup=ReplyKeyboardRemove()
    )

    return ConversationHandler.END


def error_handler(update: Update, context: CallbackContext) -> None:
    """هندلر خطاها"""
    logger.error(f"خطا {context.error} با آپدیت {update} رخ داد")

    try:
        if update and update.effective_chat:
            context.bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ متأسفانه خطایی رخ داد. لطفاً دوباره تلاش کنید یا با دستور /start از ابتدا شروع کنید."
            )
    except:
        pass


# پیکربندی و راه‌اندازی ربات
def main():
    """راه‌اندازی ربات"""
    if not config.TOKEN:
        print("❌ توکن ربات تنظیم نشده است. لطفاً ابتدا تنظیمات را انجام دهید.")
        return

    # ایجاد آپدیتر با توکن ربات
    updater = Updater(config.TOKEN, use_context=True)
    dp = updater.dispatcher

    # تعریف conversation handler اصلی
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            START: [],
            WAITING_MEMBERSHIP: [
                CallbackQueryHandler(verify_membership, pattern="^verify_membership$"),
                CommandHandler('start', start)
            ],
            RECEIVE_PHOTO: [
                MessageHandler(Filters.photo | Filters.document, receive_photo),
                CommandHandler('start', start)
            ],
            SELECT_RESIZE_METHOD: [
                CallbackQueryHandler(select_resize_method, pattern='^(resize_|compress_|start_over)'),
                CommandHandler('start', start)
            ],
            ENTER_WIDTH: [
                MessageHandler(Filters.text & ~Filters.command, enter_width),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
            ENTER_HEIGHT: [
                MessageHandler(Filters.text & ~Filters.command, enter_height),
                CallbackQueryHandler(enter_height, pattern='^(back_to_methods|start_over)'),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
            ENTER_PERCENTAGE: [
                MessageHandler(Filters.text & ~Filters.command, enter_percentage),
                CallbackQueryHandler(enter_percentage, pattern='^(back_to_methods|start_over)'),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
            SELECT_QUALITY: [
                CallbackQueryHandler(select_quality, pattern='^(quality_|back_to_methods|keep_aspect_|start_over)'),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
            ENTER_CUSTOM_QUALITY: [
                MessageHandler(Filters.text & ~Filters.command, enter_custom_quality),
                CallbackQueryHandler(enter_custom_quality, pattern='^(back_to_quality|start_over)'),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
            CONFIRM_SEND: [
                CallbackQueryHandler(confirm_send, pattern='^(send_final|change_quality|back_to_methods|start_over)'),
                CommandHandler('start', start),
                CommandHandler('cancel', cancel_command)
            ],
        },
        fallbacks=[CommandHandler('cancel', cancel_command)],
        allow_reentry=True
    )

    # تعریف conversation handler های ادمین
    add_admin_handler = ConversationHandler(
        entry_points=[CommandHandler('add_admin', add_admin_command)],
        states={
            ADMIN_SET_ADMIN: [
                MessageHandler(Filters.text | Filters.forwarded, add_admin)
            ],
        },
        fallbacks=[CommandHandler('cancel', cancel_command)]
    )

    # اضافه کردن هندلرها به دیسپچر
    dp.add_handler(conv_handler)
    dp.add_handler(add_admin_handler)
    dp.add_handler(CommandHandler('admin', admin_command))
    dp.add_handler(CommandHandler('stats', stats_command))

    # اضافه کردن هندلر خطا
    dp.add_error_handler(error_handler)

    # شروع ربات با روش پولینگ
    print(f"\n🚀 در حال راه‌اندازی ربات با روش پولینگ...")

    updater.start_polling()

    # دریافت اطلاعات ربات
    bot_info = updater.bot.get_me()
    print(f"✅ ربات {bot_info.first_name} (@{bot_info.username}) با موفقیت راه‌اندازی شد!")
    print(f"🔗 لینک ربات: https://t.me/{bot_info.username}")
    print('📝 برای توقف ربات، کلید Ctrl+C را فشار دهید.')

    # ادامه اجرای ربات
    updater.idle()


# اجرای اصلی ربات
if __name__ == '__main__':
    # تنظیم اطلاعات ربات
    setup_config()

    try:
        main()
    except KeyboardInterrupt:
        print('⛔ ربات متوقف شد.')
    except Exception as e:
        print(f'❌ خطا در اجرای ربات: {e}')

لطفاً توکن ربات خود را که از BotFather دریافت کرده‌اید وارد کنید:
··········

لطفاً آیدی عددی خود را به عنوان ادمین اصلی وارد کنید:
(برای پیدا کردن آیدی عددی خود می‌توانید به @userinfobot پیام دهید)
آیدی ادمین: 1150906790
ادمین با آیدی 1150906790 اضافه شد.

آیا می‌خواهید کانال‌های اطلاع‌رسانی را اضافه کنید؟ (بله/خیر)
بله
لطفاً آیدی کانال‌ها را وارد کنید (هر کانال در یک خط):
مثال: @mychannel
برای پایان دادن، یک خط خالی وارد کنید:
@v_pnh
کانال @v_pnh اضافه شد.
@finance_hossein
کانال @finance_hossein اضافه شد.


تنظیمات با موفقیت انجام شد.

🚀 در حال راه‌اندازی ربات با روش پولینگ...


/usr/local/lib/python3.11/dist-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn(


✅ ربات تغییر سایز و کیفیت عکس (@PhotoResizer_AA_bot) با موفقیت راه‌اندازی شد!
🔗 لینک ربات: https://t.me/PhotoResizer_AA_bot
📝 برای توقف ربات، کلید Ctrl+C را فشار دهید.


ERROR:__main__:خطا در بررسی عضویت: Member list is inaccessible
ERROR:__main__:خطا در بررسی عضویت: Member list is inaccessible
ERROR:__main__:خطای عمومی در تغییر اندازه با درصد: File of size 13815245 bytes is too big for a photo; the maximum size is 10485760 bytes
